In [1]:
print("hello")
# Go here https://python.langchain.com/v0.1/docs/get_started/quickstart/ and use the web scraping stuff to pass in the Vice article

hello


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

vice_article = "https://www.vice.com/en/article/k7zz9z/spotify-rogan-rfk-vaccine-misinformation-policy"

In [3]:
from langchain_community.document_loaders import AsyncChromiumLoader, ChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

# Load HTML
loader = ChromiumLoader([vice_article])
html = loader.load()

# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
    html, tags_to_extract=["p", "li", "div", "a"]
)

docs_transformed[0].page_content[0:500]

ImportError: cannot import name 'ChromiumLoader' from 'langchain_community.document_loaders' (C:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\fakt-ai-YW2sLaBr-py3.10\lib\site-packages\langchain_community\document_loaders\__init__.py)

In [3]:
embeddings = OpenAIEmbeddings()
loader = WebBaseLoader("https://www.vice.com/en/article/k7zz9z/spotify-rogan-rfk-vaccine-misinformation-policy")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


In [4]:
"""
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm | output_parser
a = chain.invoke({"input": "how can langsmith help with testing?"})
"""

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Answer the following question by referencing the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
"""
# Pass context directly into 'page_content'. The llm responds based only on the context provided in the page_content
a = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you create incredible videos visualize test results")]
})
"""
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Note -- 
response = retrieval_chain.invoke({"input": "List the scientific papers referenced in the provided article. Provide them in a numbered list."})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


1. "American Academy of Pediatrics has a roundup of some of the biggest [safety studies]"
2. "a large Australian study released last month found no association between COVID vaccinations and cardiac arrest"
